<a href="https://colab.research.google.com/github/jaidheer7/CSV/blob/main/PathFinder_GOL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np
import pandas as pd

from PIL import Image
import cv2

from time import perf_counter
import datetime

import os
from os.path import join
import sys


import re
import shutil

In [12]:
class newTuple(tuple):
  def __sub__(a, b):
    return (a[0] - b[0], a[1] - b[1])

In [ ]:
def createFolder(fps, grid) -> str:
    # Current Working Directory
    pdir = sys.path[0]
    
    #Getting Current Time
    curr = datetime.datetime.now()
    dateStr = str(curr.date())
    timeStr = str(curr.time())

    #Folder name
    directory = "MazeSolver_GOL_RUN_" + dateStr.replace('-','_') + "_" + timeStr.replace(':', '_')
    
    #Final Path
    fpath = join(pdir, directory)
    
    #Creating the directory
    try:
      os.mkdir(fpath)
    except Exception as exp:
      return
    
    # Creating the Images SubFolder
    imgPath = join(fpath, 'Images')
    os.mkdir(imgPath)

    # Creating CSVs SubFolder
    csvPath = join(fpath, 'CSVs')
    os.mkdir(csvPath)
    
    # Writing usefull files
    
    # Saving the inital grid state
    np.savetxt(fname = join(fpath, 'initalGrid.csv'), X = grid, delimiter = ',', fmt = '%d')
    
    info = """
           author: S.Jaidheer
           github: JAIDHEER007
           
           Implementing Path Finder using Python
           
           INFO
           =====================================================
           Date: {fdate}
           time: {ftime}
            
           Rows: {frows}
           Cols: {fcols}
           
           Generations: {fgen}
           Set FPS: {ffps}
           
           """.format(fdate = dateStr, ftime = timeStr, 
                      frows = grid.shape[0], fcols = grid.shape[1], 
                      ffps = fps)
           
    with open(join(fpath, 'info.txt'), 'w') as fileHandle:
        fileHandle.writelines(info)
           
    return fpath

In [15]:
directionMatrix = [newTuple((i, j)) for i in [0, 1, -1] for j in [0, 1, -1] if abs(i) != abs(j)]

In [21]:
class MazeSolver:
  def isValid(shape, coordinates) -> bool:
    rows, cols = shape
    i, j = coordinates
    return (i >= 0) and (i < rows) and (j >= 0) and (j < cols)
  
  def dfs(grid, coordinates):
    # rows, cols = grid.shape

    if not isValid(grid.shape, coordinates) or grid[coordinates] in [0, 2, 4]: return
    if grid[coordinates] == '3':
      print(coordinates)
      return
    grid[coordinates] = 4

    dfs(grid, coordinates + directionMatrix[0])
    dfs(grid, coordinates + directionMatrix[1])
    dfs(grid, coordinates + directionMatrix[2])
    dfs(grid, coordinates + directionMatrix[3])
    

  def __init__(self, startGrid, generations, startCoordinates, fps = 35):
    self.__rows, self.__cols = startGrid.shape 
    self.__fps = fps
    self.__grid = startGrid
    self.__start = startCoordinates



    #Folder Path
    self.__fpath = createFolder(fps = self.__fps, 
                                grid = self.__grid)
    


Reading CSV from github

In [8]:
url = 'https://raw.githubusercontent.com/jaidheer7/CSV/main/State19.csv'
grid = pd.read_csv(url, header=None, dtype=int)

In [9]:
grid.shape

(200, 200)

In [18]:
startingLocation = newTuple((0, 0))
endingLocation = newTuple((0, 0))

In [24]:
if not MazeSolver.isValid(grid.shape, startingLocation):
  raise Exception("Starting Location Outside the grid")
if not MazeSolver.isValid(grid.shape, endingLocation):
  raise Exception("Ending Location Outside the grid")